In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle as pkl
import os 

from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from midiutil import MIDIFile

In [2]:
class MelodyRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MelodyRNN, self).__init__()

        self.embedding = nn.Embedding(input_size, hidden_size, padding_idx=0)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, input_size)
        self.dropout = nn.Dropout(0.3)
        self.hidden = Variable(torch.zeros(1, 1, hidden_size))                
        
    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.dropout(embedded)
        output, self.hidden = self.gru(embedded, self.hidden)
        output = self.out(self.hidden[-1])
        return output, self.hidden

In [3]:
model = torch.load('../objects/Full_RNN')
index2chord = pkl.load( open( "../objects/index2chord.pkl", "rb" ) )

/anaconda3/envs/general/lib/python3.7/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/anaconda3/envs/general/lib/python3.7/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.GRU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/anaconda3/envs/general/lib/python3.7/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the obje

In [4]:
def pick_rand_note(out):
    sorted_pred = out.argsort()
    pred = np.random.choice(sorted_pred[0][-10:].detach().numpy(), 1)[0]
    return pred

def generate(model, improv=0.5):
    pred = 0
    model.eval()
    notes = []
    all_notes = torch.tensor([[0,0,1]])
    for i in range(96):
        out, _ = model(all_notes)
        if np.random.binomial(n=1, p=improv) == 1:
            pred = pick_rand_note(out)
        else:
            pred = int(out.argmax())
        
        # Ensure notes are chosen
        while pred == 0 or pred == 1 or pred == 2:
            pred = pick_rand_note(out)
            
        all_notes = torch.cat([all_notes, torch.tensor([[pred]])], 1)
    all_notes = torch.cat([all_notes[0], torch.tensor([2])])
    converted_notes = [index2chord[int(note)] for note in all_notes]
    return converted_notes

def generate2(model, improv=0.5):
    pred = 0
    model.eval()
    notes = []
    all_notes = torch.tensor([[0,0,1]])
    
    # Generate part I
    part_I = []
    for i in range(16):
        out, _ = model(all_notes)
        if np.random.binomial(n=1, p=improv) == 1:
            pred = pick_rand_note(out)
        else:
            pred = int(out.argmax())
        
        # Ensure notes are chosen
        while pred == 0 or pred == 1 or pred == 2:
            pred = pick_rand_note(out)
            
        part_I.append(pred)
        all_notes = torch.cat([all_notes, torch.tensor([[pred]])], 1)
    # Repeat part I 
    all_notes = torch.cat([all_notes, torch.tensor([part_I])], 1)
    # Generate part IV
    part_IV = []
    for i in range(16):
        out, _ = model(all_notes)
        if np.random.randint(2) == 1:
            pred = pick_rand_note(out)
        else:
            pred = int(out.argmax())
        
        # Ensure notes are chosen
        while pred == 0 or pred == 1 or pred == 2:
            pred = pick_rand_note(out)
            
        part_IV.append(pred)
        all_notes = torch.cat([all_notes, torch.tensor([[pred]])], 1)
    # Repeat part I
    all_notes = torch.cat([all_notes, torch.tensor([part_I])], 1)
    # Generate part V
    part_V = []
    for i in range(16):
        out, _ = model(all_notes)
        if np.random.randint(2) == 1:
            pred = pick_rand_note(out)
        else:
            pred = int(out.argmax())
        
        # Ensure notes are chosen
        while pred == 0 or pred == 1 or pred == 2:
            pred = pick_rand_note(out)
            
        part_V.append(pred)
        all_notes = torch.cat([all_notes, torch.tensor([[pred]])], 1) 
    # Repeat part I
    all_notes = torch.cat([all_notes, torch.tensor([part_I])], 1)   
    # Append ending token
    all_notes = torch.cat([all_notes[0], torch.tensor([2])])
    converted_notes = [index2chord[int(note)] for note in all_notes]
    return converted_notes
    
note_mapper = {'C': 60, 'Db':61, 'D':62,
               'Eb':63, 'E':64, 'F':65,
              'Gb':66, 'G':67, 'Ab':68,
              'A':69, 'Bb': 70, 'B':71, 'R':None}

In [45]:
def generate_melody(output_file, gen_song=False, improv=0.5):
    os.system('rm %s' %output_file)
    melody = None
    if gen_song:
        melody = generate2(model, improv)[:-1]        
    else:
        melody = generate(model, improv)[:-1]
    notes = [note_mapper[each.split('_')[0]] for each in melody[3:]]
    lengths = [each.split('_')[1] for each in melody[3:]]    
    track    = 0
    channel  = 0
    time     = 0   # In beats
    duration = 0.5   # In beats
    tempo    = 160  # In BPM
    volume   = 100 # 0-127, as per the MIDI standard

    MyMIDI = MIDIFile(1) # One track, defaults to format 1 (tempo track
                         # automatically created)
    MyMIDI.addTempo(track,time, tempo)

    prev_pitch = None
    long_note = False
    some_notes = []

    for i, pitch in enumerate(notes):
        if pitch is None:
            if prev_pitch is not None and long_note:
                some_notes.append((time,prev_pitch, duration))
                MyMIDI.addNote(track, channel, prev_pitch, time, duration, volume)
                time += duration
                duration = 0.5
            prev_pitch = pitch
            time += 0.5

        elif long_note and prev_pitch != pitch and prev_pitch is not None:
            some_notes.append((time,prev_pitch, duration))  
            MyMIDI.addNote(track, channel, prev_pitch, time, duration, volume)

            time += duration
            duration = 0.5
            if lengths[i] == 'L':
                some_notes.append((time, pitch, duration))              
                MyMIDI.addNote(track, channel, pitch, time, duration, volume)
                long_note = False
                time += 0.5

        elif long_note and prev_pitch == pitch:
            duration += 0.5    

            if lengths[i] == 'S':
                long_note = True
            elif lengths[i] == 'L':
                some_notes.append((time, pitch, duration))            
                MyMIDI.addNote(track, channel, pitch, time, duration, volume)  
                long_note = False
                time += duration            
                duration = 0.5

        elif not long_note:
            if lengths[i] == 'S':
                long_note = True
                prev_pitch = pitch
            else:
                some_notes.append((time, pitch, duration))            
                MyMIDI.addNote(track, channel, pitch, time, duration, volume)
                time += 0.5
    
        prev_pitch = pitch
    with open(output_file, "wb") as output_file:
        MyMIDI.writeFile(output_file)

In [80]:
generate_melody('../generated_songs/my_song.midi', gen_song=True, improv=0.5)

96


In [81]:
generate_melody('../generated_songs/my_song2.midi', gen_song=True, improv=0.1)

96


In [82]:
generate_melody('../generated_songs/my_song3.midi', gen_song=True, improv=0.9)

96


In [83]:
generate_melody('../generated_songs/my_improv.midi', gen_song=False, improv=0.5)

96


In [84]:
generate_melody('../generated_songs/my_improv2.midi', gen_song=False, improv=0.1)

96


In [85]:
generate_melody('../generated_songs/my_improv3.midi', gen_song=False, improv=0.9)

96
